In [3]:
#!/usr/bin/env python3
"""
Task 1 - Data Exploration Script
Loads and explores the Brent Oil Price dataset to understand schema, content, and relationships.
"""

import pandas as pd
import numpy as np
from pathlib import Path
import sys

def main():
    """Main exploration function"""
    
    # ====== 1. Configuration ======
    print("=" * 80)
    print("BRENT OIL PRICE DATA EXPLORATION SCRIPT")
    print("=" * 80)
    
    DATA_DIR = Path("../data/raw/")
    
    # Check if data directory exists
    if not DATA_DIR.exists():
        print(f"ERROR: Data directory not found at {DATA_DIR}")
        print("Please adjust the DATA_DIR path or create the data directory.")
        sys.exit(1)
    
    # ====== 2. Load datasets ======
    print("\n" + "=" * 80)
    print("LOADING DATASETS")
    print("=" * 80)
    
    try:
        # Load Brent Oil Prices CSV
        oil_file = DATA_DIR / "BrentOilPrices.csv"
        
        if oil_file.exists():
            print("Loading Brent Oil Prices data...")
            oil_df = pd.read_csv(oil_file)
            print(f"✓ Loaded BrentOilPrices.csv ({len(oil_df)} rows)")
            
            # For compatibility with original script structure
            data_df = oil_df.copy()
            impact_links_df = pd.DataFrame()  # Empty for now
            reference_codes_df = pd.DataFrame()  # Empty for now
        else:
            print(f"ERROR: Data file not found in {DATA_DIR}")
            print(f"Expected: {oil_file.name}")
            sys.exit(1)
        
    except Exception as e:
        print(f"ERROR loading data: {e}")
        sys.exit(1)
    
    # ====== 3. Explore schema ======
    print("\n" + "=" * 80)
    print("DATA SCHEMA")
    print("=" * 80)
    
    print("\n--- BRENT OIL PRICES DATASET STRUCTURE ---")
    print(f"Total records: {len(data_df):,}")
    print(f"Columns: {len(data_df.columns)}")
    print("\nColumn names:")
    for i, col in enumerate(data_df.columns, 1):
        print(f"  {i:2d}. {col}")
    
    print("\n" + "-" * 40)
    print("Data types:")
    print(data_df.dtypes.to_string())
    
    # ====== 4. Show sample data ======
    print("\n" + "=" * 80)
    print("SAMPLE DATA")
    print("=" * 80)
    
    print("\nFirst 10 rows:")
    print(data_df.head(10).to_string())
    
    print("\nLast 10 rows:")
    print(data_df.tail(10).to_string())
    
    # ====== 5. Basic statistics ======
    print("\n" + "=" * 80)
    print("BASIC STATISTICS")
    print("=" * 80)
    
    print("\nDescriptive statistics:")
    print(data_df.describe().to_string())
    
    # ====== 6. Check for missing values ======
    print("\n" + "=" * 80)
    print("MISSING VALUE ANALYSIS")
    print("=" * 80)
    
    missing_values = data_df.isnull().sum()
    total_cells = np.prod(data_df.shape)
    total_missing = missing_values.sum()
    
    print(f"\nTotal cells in dataset: {total_cells:,}")
    print(f"Total missing values: {total_missing:,} ({total_missing/total_cells*100:.2f}%)")
    
    if total_missing > 0:
        print("\nMissing values by column:")
        for column, missing_count in missing_values[missing_values > 0].items():
            percentage = (missing_count / len(data_df)) * 100
            print(f"  {column:20s}: {missing_count:6,} ({percentage:.2f}%)")
    
    # ====== 7. Temporal coverage analysis ======
    print("\n" + "=" * 80)
    print("TEMPORAL COVERAGE")
    print("=" * 80)
    
    # Check for date-related columns
    date_columns = [col for col in data_df.columns if 'date' in col.lower() or 'time' in col.lower() or 'year' in col.lower() or 'month' in col.lower()]
    
    if date_columns:
        print(f"\nDate-related columns found: {date_columns}")
        
        for date_col in date_columns:
            print(f"\n--- Analyzing {date_col} ---")
            
            # Try to convert to datetime
            try:
                data_df[f'{date_col}_parsed'] = pd.to_datetime(data_df[date_col], errors='coerce')
                valid_dates = data_df[f'{date_col}_parsed'].dropna()
                
                if len(valid_dates) > 0:
                    print(f"Valid dates: {len(valid_dates):,} ({len(valid_dates)/len(data_df)*100:.1f}%)")
                    print(f"Date range: {valid_dates.min().date()} to {valid_dates.max().date()}")
                    print(f"Timespan: {(valid_dates.max() - valid_dates.min()).days / 365.25:.1f} years")
                    
                    # Check for duplicates or gaps
                    date_counts = valid_dates.value_counts().sort_values(ascending=False)
                    duplicates = date_counts[date_counts > 1]
                    if len(duplicates) > 0:
                        print(f"⚠️  {len(duplicates):,} dates have duplicate entries")
                    
                    # Check for gaps
                    date_range = pd.date_range(start=valid_dates.min(), end=valid_dates.max())
                    missing_dates = date_range.difference(valid_dates)
                    if len(missing_dates) > 0:
                        print(f"⚠️  {len(missing_dates):,} dates missing from continuous range")
                    
                    # Yearly distribution
                    print("\nRecords per year:")
                    yearly_counts = valid_dates.dt.year.value_counts().sort_index()
                    for year, count in yearly_counts.items():
                        print(f"  {year}: {count:6,}")
                        
            except Exception as e:
                print(f"Could not parse {date_col} as datetime: {e}")
    else:
        # Try to identify any column that might be a date
        print("\nNo obvious date columns found. Checking all columns...")
        for col in data_df.columns:
            sample = data_df[col].dropna().head(5).astype(str).tolist()
            # Simple heuristic: if any sample looks like a date
            date_like = any('/' in str(s) or '-' in str(s) for s in sample if len(str(s)) > 6)
            if date_like:
                print(f"  {col} might be a date column (sample: {sample})")
    
    # ====== 8. Price/Value analysis ======
    print("\n" + "=" * 80)
    print("PRICE/VALUE ANALYSIS")
    print("=" * 80)
    
    # Look for columns that might contain price/value data
    value_keywords = ['price', 'value', 'close', 'open', 'high', 'low', 'volume', 'amount']
    value_columns = [col for col in data_df.columns 
                    if any(keyword in col.lower() for keyword in value_keywords)]
    
    if value_columns:
        print(f"\nPrice/value-related columns found: {value_columns}")
        
        for value_col in value_columns:
            print(f"\n--- Analyzing {value_col} ---")
            
            # Convert to numeric if possible
            data_df[f'{value_col}_numeric'] = pd.to_numeric(data_df[value_col], errors='coerce')
            numeric_values = data_df[f'{value_col}_numeric'].dropna()
            
            if len(numeric_values) > 0:
                print(f"Numeric values: {len(numeric_values):,} ({len(numeric_values)/len(data_df)*100:.1f}%)")
                print(f"Min: {numeric_values.min():.2f}")
                print(f"Max: {numeric_values.max():.2f}")
                print(f"Mean: {numeric_values.mean():.2f}")
                print(f"Median: {numeric_values.median():.2f}")
                print(f"Std Dev: {numeric_values.std():.2f}")
                
                # Check for outliers (using IQR method)
                Q1 = numeric_values.quantile(0.25)
                Q3 = numeric_values.quantile(0.75)
                IQR = Q3 - Q1
                lower_bound = Q1 - 1.5 * IQR
                upper_bound = Q3 + 1.5 * IQR
                outliers = numeric_values[(numeric_values < lower_bound) | (numeric_values > upper_bound)]
                
                if len(outliers) > 0:
                    print(f"⚠️  {len(outliers):,} potential outliers found ({len(outliers)/len(numeric_values)*100:.2f}%)")
                
                # Check for zero or negative values if that's unexpected
                non_positive = numeric_values[numeric_values <= 0]
                if len(non_positive) > 0:
                    print(f"⚠️  {len(non_positive):,} non-positive values found")
    
    # ====== 9. Correlation analysis ======
    print("\n" + "=" * 80)
    print("CORRELATION ANALYSIS")
    print("=" * 80)
    
    # Identify numeric columns for correlation
    numeric_cols = data_df.select_dtypes(include=[np.number]).columns.tolist()
    
    if len(numeric_cols) > 1:
        print(f"\nNumeric columns available for correlation: {numeric_cols}")
        
        correlation_matrix = data_df[numeric_cols].corr()
        
        print("\nCorrelation matrix:")
        print(correlation_matrix.to_string())
        
        # Find highly correlated pairs
        print("\nHighly correlated pairs (|r| > 0.7):")
        for i in range(len(numeric_cols)):
            for j in range(i+1, len(numeric_cols)):
                corr = correlation_matrix.iloc[i, j]
                if abs(corr) > 0.7:
                    print(f"  {numeric_cols[i]} - {numeric_cols[j]}: {corr:.3f}")
    else:
        print("\nNot enough numeric columns for correlation analysis")
    
    # ====== 10. Data quality checks ======
    print("\n" + "=" * 80)
    print("DATA QUALITY CHECKS")
    print("=" * 80)
    
    print("\n1. Duplicate Records:")
    duplicates = data_df.duplicated().sum()
    if duplicates > 0:
        print(f"⚠️  {duplicates:,} duplicate records found ({duplicates/len(data_df)*100:.2f}%)")
    else:
        print("✓ No duplicate records found")
    
    print("\n2. Inconsistent Data Types:")
    for col in data_df.columns:
        unique_types = data_df[col].apply(type).nunique()
        if unique_types > 1:
            print(f"⚠️  {col} has {unique_types} different data types")
    
    print("\n3. Unusual Values:")
    # Check for extreme values in numeric columns
    for col in numeric_cols:
        if len(data_df[col].dropna()) > 0:
            mean_val = data_df[col].mean()
            std_val = data_df[col].std()
            extreme_threshold = mean_val + 5 * std_val
            extreme_values = data_df[data_df[col] > extreme_threshold]
            if len(extreme_values) > 0:
                print(f"⚠️  {col} has {len(extreme_values)} values > 5 std dev from mean")
    
    # ====== 11. Save summary outputs ======
    print("\n" + "=" * 80)
    print("SAVING SUMMARY OUTPUTS")
    print("=" * 80)
    
    # Create summary statistics
    summary_stats = {
        "total_records": len(data_df),
        "total_columns": len(data_df.columns),
        "total_missing_values": total_missing,
        "missing_percentage": total_missing/total_cells*100,
        "duplicate_records": duplicates,
        "date_columns_found": len(date_columns),
        "numeric_columns_found": len(numeric_cols),
        "value_columns_found": len(value_columns)
    }
    
    # Add column-specific info
    for col in data_df.columns:
        summary_stats[f"{col}_dtype"] = str(data_df[col].dtype)
        summary_stats[f"{col}_unique"] = data_df[col].nunique()
        summary_stats[f"{col}_missing"] = data_df[col].isnull().sum()
    
    # Create summary DataFrame
    summary_df = pd.DataFrame.from_dict(
        summary_stats, 
        orient='index', 
        columns=['value']
    )
    
    # Save to CSV
    output_dir = Path("../results/reports/")
    output_dir.mkdir(parents=True, exist_ok=True)
    
    output_file = output_dir / "task1_oil_data_summary.csv"
    summary_df.to_csv(output_file)
    print(f"\n✓ Summary statistics saved to: {output_file}")
    
    # Save data samples
    sample_file = output_dir / "task1_data_samples.csv"
    pd.concat([data_df.head(20), data_df.tail(20)]).to_csv(sample_file, index=False)
    print(f"✓ Data samples saved to: {sample_file}")
    
    # Save missing values report
    if total_missing > 0:
        missing_report = pd.DataFrame({
            'column': missing_values.index,
            'missing_count': missing_values.values,
            'missing_percentage': (missing_values.values / len(data_df)) * 100
        })
        missing_report = missing_report[missing_report['missing_count'] > 0].sort_values('missing_percentage', ascending=False)
        missing_file = output_dir / "task1_missing_values_report.csv"
        missing_report.to_csv(missing_file, index=False)
        print(f"✓ Missing values report saved to: {missing_file}")
    
    # ====== 12. Key findings summary ======
    print("\n" + "=" * 80)
    print("KEY FINDINGS SUMMARY")
    print("=" * 80)
    
    print("\n1. DATASET OVERVIEW:")
    print(f"   • {summary_stats['total_records']:,} total records")
    print(f"   • {summary_stats['total_columns']:,} columns")
    
    print("\n2. DATA QUALITY:")
    print(f"   • {summary_stats['total_missing_values']:,} missing values ({summary_stats['missing_percentage']:.2f}%)")
    print(f"   • {summary_stats['duplicate_records']:,} duplicate records")
    
    print("\n3. TEMPORAL COVERAGE:")
    if date_columns:
        print(f"   • {len(date_columns)} date-related column(s) found")
        # Add specific date info if available
        for date_col in date_columns:
            if f'{date_col}_parsed' in data_df.columns:
                valid_dates = data_df[f'{date_col}_parsed'].dropna()
                if len(valid_dates) > 0:
                    print(f"   • {date_col}: {valid_dates.min().date()} to {valid_dates.max().date()}")
    else:
        print("   • No obvious date columns identified")
    
    print("\n4. PRICE/VALUE DATA:")
    if value_columns:
        print(f"   • {len(value_columns)} value-related column(s) found")
        for value_col in value_columns:
            print(f"   • {value_col}: available for analysis")
    else:
        print("   • No obvious price/value columns identified")
    
    print("\n5. FORECASTING POTENTIAL:")
    if date_columns and value_columns:
        print("   ✓ Dataset appears suitable for time series forecasting")
        print("   • Contains both temporal and value dimensions")
    else:
        print("   ⚠️  Dataset may need preprocessing for time series analysis")
    
    print("\n" + "=" * 80)
    print("EXPLORATION COMPLETE - Ready for Task 1 README")
    print("=" * 80)
    
    # Print next steps
    print("\nNEXT STEPS:")
    print("1. Review the generated reports in ../results/reports/")
    print("2. Write Task 1 README based on these findings")
    print("3. Consider what data enrichment might be needed")
    print("4. Plan preprocessing steps for time series analysis")

if __name__ == "__main__":
    main()

BRENT OIL PRICE DATA EXPLORATION SCRIPT

LOADING DATASETS
Loading Brent Oil Prices data...
✓ Loaded BrentOilPrices.csv (9011 rows)

DATA SCHEMA

--- BRENT OIL PRICES DATASET STRUCTURE ---
Total records: 9,011
Columns: 2

Column names:
   1. Date
   2. Price

----------------------------------------
Data types:
Date         str
Price    float64

SAMPLE DATA

First 10 rows:
        Date  Price
0  20-May-87  18.63
1  21-May-87  18.45
2  22-May-87  18.55
3  25-May-87  18.60
4  26-May-87  18.63
5  27-May-87  18.60
6  28-May-87  18.60
7  29-May-87  18.58
8  01-Jun-87  18.65
9  02-Jun-87  18.68

Last 10 rows:
              Date  Price
9001  Nov 01, 2022  95.12
9002  Nov 02, 2022  96.07
9003  Nov 03, 2022  95.29
9004  Nov 04, 2022  99.53
9005  Nov 07, 2022  99.87
9006  Nov 08, 2022  96.85
9007  Nov 09, 2022  93.05
9008  Nov 10, 2022  94.25
9009  Nov 11, 2022  96.37
9010  Nov 14, 2022  93.59

BASIC STATISTICS

Descriptive statistics:
             Price
count  9011.000000
mean     48.420782
std 

C:\Users\321\AppData\Local\Temp\ipykernel_15272\1020993614.py:124: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data_df[f'{date_col}_parsed'] = pd.to_datetime(data_df[date_col], errors='coerce')


Valid dates: 9,011 (100.0%)
Date range: 1987-05-20 to 2022-11-14
Timespan: 35.5 years
⚠️  3,952 dates missing from continuous range

Records per year:
  1987:    160
  1988:    255
  1989:    254
  1990:    256
  1991:    257
  1992:    257
  1993:    252
  1994:    252
  1995:    253
  1996:    254
  1997:    248
  1998:    253
  1999:    249
  2000:    253
  2001:    257
  2002:    255
  2003:    258
  2004:    261
  2005:    257
  2006:    255
  2007:    250
  2008:    253
  2009:    252
  2010:    252
  2011:    248
  2012:    249
  2013:    252
  2014:    254
  2015:    255
  2016:    255
  2017:    256
  2018:    252
  2019:    258
  2020:    256
  2021:    253
  2022:    220

PRICE/VALUE ANALYSIS

Price/value-related columns found: ['Price']

--- Analyzing Price ---
Numeric values: 9,011 (100.0%)
Min: 9.10
Max: 143.95
Mean: 48.42
Median: 38.57
Std Dev: 32.86

CORRELATION ANALYSIS

Numeric columns available for correlation: ['Price', 'Price_numeric']

Correlation matrix:
        